In [1]:
import os
import pandas as pd
import numpy.random as rnd
import warnings
import json
import sqlalchemy as sql
import datetime
import yfinance as yf
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
#from MCForecastTools import MCSimulation
from bokeh.plotting import figure, output_file
from bokeh.resources import INLINE

In [17]:
# Date range
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*1) #trading days(252) * amount of years to go back 

start = start_date.strftime("%Y-%m-%d")
end = today.strftime("%Y-%m-%d")

# Tickers of assets
assets = ['NOC']
assets.sort()

# Downloading data
og_data1 = yf.download(assets, start = start, end = end)
#data = og_data.loc[:,('Adj Close', slice(None))]
#data.columns = assets

#Y = data[assets].pct_change().dropna()
display(og_data1)

display(start)
display(end)
#display(Y)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-08-16,485.329987,488.600006,483.230011,485.500000,478.375000,500800
2022-08-17,482.720001,496.299988,482.630005,492.510010,485.282166,852000
2022-08-18,493.989990,497.200012,490.429993,492.880005,485.646729,592000
2022-08-19,492.130005,493.239990,487.839996,490.750000,483.547974,606600
2022-08-22,488.000000,493.140015,483.739990,484.190002,477.084229,607700
...,...,...,...,...,...,...
2023-08-09,433.000000,437.570007,432.459991,435.549988,435.549988,1547800
2023-08-10,434.350006,434.739990,429.640015,430.269989,430.269989,800500
2023-08-11,430.269989,432.660004,429.670013,431.049988,431.049988,733800


'2022-08-16'

'2023-08-16'

In [8]:
# Date range
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*1) #trading days(252) * amount of years to go back 

start = start_date.strftime("%Y-%m-%d")
end = today.strftime("%Y-%m-%d")

# Tickers of assets
assets = ['NOC', 'F', 'JNJ', 'ZTS', 'VTR', 'MRK', 'MO', 'KO', 'DAR', 'THO',
       'T', 'TMUS', 'APTV', 'PLD', 'COIN', 'ABBV', 'TEL', 'TSN', 'URI',
       'UNH', 'EQR', 'EPRT', 'ESS', 'EVBG', 'ES', 'EXC', 'ADC', 'MOS',
       'EXR', 'MOH', 'MRNA', 'FN', 'FERG', 'KDP', 'KRC', 'SGH', 'AYX',
       'FIS', 'FNF', 'ARE', 'ALEX', 'FCPT', 'KRG', 'KR', 'GTY', 'IOT',
       'GILD', 'RTX', 'KLIC', 'HALO', 'PEAK', 'RLJ', 'HEI-A', 'WPM',
       'MAA', 'SYY', 'WELL', 'TNDM', 'SKT', 'PK', 'TFX', 'ROIC', 'RLAY',
       'RWT', 'RBA', 'VZ', 'KHC', 'PARAA', 'LHX', 'SCHW', 'UBA', 'HST',
       'MELI', 'QDEL', 'HHC', 'UE', 'AAP', 'TMO', 'PSA', 'MMM', 'LBTYA',
       'ICHR', 'IRT', 'MKTX', 'AMCX', 'AMCR', 'CBU', 'LBTYK', 'AMT', 'CB',
       'MTB', 'LVMUY', 'CIEN', 'LSI', 'CNC', 'SR', 'BGNE', 'LMT', 'STT',
       'BELFB', 'IIPR', 'PODD', 'CADE', 'CTRE', 'CFG', 'IFF', 'OGS',
       'INVH', 'BXP', 'NVO', 'IRM', 'TRIP', 'TFC', 'TEAM', 'AVB', 'PD',
       'CAG', 'BMY', 'ADM', 'CHPT', 'BEPC', 'ANET', 'JAZZ', 'CCI', 'CUBE',
       'SHO', 'NXRT', 'PGR', 'JBGS', 'PLTK', 'PHIN', 'SBGI', 'DRH', 'DG',
       'DLTR', 'PECO', 'BFS', 'SBAC', 'PFE', 'CLVT', 'PEB', 'STX', 'PYPL',
       'D', 'SLG', 'EIX', 'USB', 'ENTA', 'EPAM', 'NSA', 'INN', 'UDR',
       'UPS', 'EPR', 'EQIX', 'AVTR', 'YUMC', 'LUMN', 'OLCLY', 'AMADY',
       'BUD', 'ANZGY', 'AON', 'ARGX', 'CJPRY', 'SSREY', 'ARKAY', 'AHKSY',
       'ASHTY', 'ASML', 'ASAZY', 'ALPMY', 'AZN', 'ATLKY', 'AVVIY',
       'AXAHY', 'BAESY', 'BBVA', 'SAN', 'BCS', 'CLLNY', 'BAYRY', 'CZMWY',
       'BDRFY', 'CCOEY', 'BNPQY', 'SWRAY', 'BDNNY', 'CGEMY', 'BXBLY',
       'BNTGY', 'CAJPY', 'BRDCY', 'BTI', 'BTLCY', 'BVVBY', 'BURBY',
       'PRNDY', 'PANDY', 'SCGLY', 'SOBKY', 'SFTBY', 'SKHHY', 'SONVY',
       'SONY', 'SPKKY', 'SSEZY', 'STLA', 'OVCHY', 'OTSKY', 'DNNGY', 'STM',
       'OPHLY', 'OMRNY', 'OLYMY', 'OCDDY', 'NVS', 'ZURVY', 'ZLNDY',
       'NRDBY', 'NRILY', 'NMR', 'NOK', 'NNGRY', 'NDEKY', 'NSANY', 'NTTYY',
       'NTDOY', 'NINOY', 'NJDCY', 'NICE', 'SKFRY', 'SPXCY', 'NEXOY',
       'NCMGY', 'SOMMY', 'YASKY', 'SINGY', 'YARIY', 'YAMHY', 'WPP',
       'WRDLY', 'NTES', 'SBGSY', 'SE', 'SKLTY', 'SKHSY', 'SVNDY', 'SGSOY',
       'SMNNY', 'NSRGY', 'NWG', 'NGG', 'NABZY', 'SHECY', 'NCBDY', 'SGIOY',
       'MRAAY', 'MURGY', 'MTUAY', 'MSADY', 'SSDOY', 'SIEGY', 'MHGVY',
       'SMNEY', 'SXYAY', 'MFG', 'SMMNY', 'SMCAY', 'SNN', 'SMFKY', 'SOAGY',
       'SAP', 'MSLOY', 'MTSFY', 'MUFG', 'MIELY', 'MSSMY', 'SNY', 'SDVKY',
       'SAXPY', 'SALRY', 'SAFRY', 'RPT', 'KKPNY', 'RHHBY', 'RIO', 'WJRYY',
       'VLVLY', 'VWAGY', 'VWAPY', 'RNMBY', 'VOD', 'VCISY', 'RTO', 'RELX',
       'RANJY', 'MGDDY', 'VEOEY', 'VACNY', 'MKKGY', 'MBGYY', 'QBIEY',
       'QABSY', 'PUBGY', 'PRYMY', 'MDIBY', 'MZDAY', 'MKTAY', 'PUK',
       'PROSY', 'MQBKY', 'MTHRY', 'LRLCY', 'LZAGY', 'LOGI', 'LYG',
       'JSGRY', 'BMRRY', 'AMKBY', 'ABBNY', 'AAVMY', 'AHEXY', 'ADDYY',
       'ATEYY', 'LGRDY', 'LSRCY', 'AIQUY', 'KHNGY', 'KUBTY', 'KSRYY',
       'ADRNY', 'KMTUY', 'KNRRY', 'KNBWY', 'KGSPY', 'KGFHY', 'KIM',
       'ADYEY', 'KRYAY', 'UPMMY', 'PPRUY', 'KDDIY', 'AEG', 'ANYYY',
       'AONNY', 'AAGIY', 'EADSY', 'AJINY', 'AKZOY', 'ALC', 'KBCSY',
       'JTKWY', 'UUGRY', 'UOVEY', 'UL', 'UMICY', 'UCBJY', 'UBS', 'JRONY',
       'JAPAY', 'JHX', 'TMICY', 'TT', 'TM', 'TOTDY', 'TRYIY', 'JSAIY',
       'ITTOY', 'ISUZY', 'ISNPY', 'IHG', 'ING', 'IFJPY', 'IFNNY', 'IDEXY',
       'TOELY', 'TKOMY', 'IMBBY', 'ICL', 'IBDRY', 'HSQVY', 'TKAMY',
       'HSBC', 'HOCPY', 'TRUMY', 'TEZNY', 'TCEHY', 'TMSNY', 'TLGPY',
       'TLPFY', 'TEF', 'TTNDY', 'TTDKY', 'TAK', 'TSM', 'SSMXY', 'SYIEY',
       'SCMWY', 'HKXCY', 'HMC', 'HCMLY', 'HTHIY', 'HXGBY', 'HESAY',
       'HEINY', 'HDELY', 'SMTOY', 'SMFG', 'HDB', 'HVRRY', 'HLN', 'GSK',
       'GVDNY', 'GNGBY', 'GMAB', 'GBERY', 'GASNY', 'FJTSY', 'FUJIY',
       'FMS', 'FSUGY', 'FBP', 'ALFVY', 'FRRVY', 'RACE', 'ALLE', 'ALIZY',
       'FRCOY', 'FANUY', 'ESLOY', 'EBKDY', 'ERIC', 'EPOKY', 'SUTNY',
       'ENGIY', 'EDPFY', 'ENLAY', 'DRPRY', 'DQJCY', 'DNBBY', 'DSCSY',
       'DEO', 'DTEGY', 'DHLGY', 'SUHJY', 'DBOEY', 'DB', 'DNTUY', 'STBFY',
       'DNZOY', 'DBSDY', 'DASTY', 'DANOY', 'DWAHY', 'DTRUY', 'DKILY',
       'DSNKY', 'DLICY', 'DFKCY', 'DNPLY', 'CSLLY', 'CRH', 'CRARY',
       'CMSQY', 'CODYY', 'CMWAY', 'CRZBY', 'CLPBY', 'CHEOY', 'CHGCY',
       'SZKMY', 'SVNLY', 'SWDBY', 'VAL', 'OLPX', 'PARA', 'HAL', 'QS',
       'CHX', 'CLF', 'DISH', 'ATUS', 'LCID', 'FTV', 'IR', 'WFAFY', 'AIG',
       'YOU', 'CTSH', 'COLB', 'CONN', 'CSGP', 'CWK', 'CVS', 'DDOG', 'DE',
       'DLR', 'DNB', 'EWBC', 'EDIT', 'ESRT', 'ENPH', 'EL', 'FITB',
       'FCNCA', 'FLO', 'FCEL', 'GM', 'GTLB', 'HQY', 'HRL', 'HUN', 'INSP',
       'IQV', 'JKHY', 'JCI', 'JLL', 'JPM', 'KRTX', 'LC', 'FWONK', 'LNC',
       'LPSN', 'MDGL', 'MRVI', 'VAC', 'MPW', 'MSFT', 'MLKN', 'MRTX',
       'TAP', 'MDLZ', 'NDAQ', 'NTRA', 'NFG', 'NYCB', 'NEM', 'NOV', 'NTNX',
       'NVDA', 'OPEN', 'OUT', 'PLUG', 'QTWO', 'RGA', 'RNG', 'RITM',
       'ROKU', 'SBRA', 'SRPT', 'SHLS', 'SWX', 'SYF', 'TEX', 'TSLA',
       'UGI', 'UAL', 'X', 'UNM', 'VCYT', 'VTRS', 'VNO', 'W', 'WBS', 'WFC',
       'WY', 'WTW', 'ZS', 'BAC', 'AKR', 'ACN', 'AFRM', 'AGNC', 'ALB',
       'AMZN', 'AAT', 'FOLD', 'APO', 'AAPL', 'ARRY', 'CAR', 'AXS', 'BILL',
       'CPT', 'COF', 'CAT', 'GTLS', 'ASXFY', 'BHKLY', 'NTDTY', 'MONOY',
       'GRFS', 'DPSGY', 'CNHI', 'MITEY', 'SKBSY', 'PM',
       'CTLT', 'CPB', 'OHI', 'LLY', 'WM', 'SNOW', 'DAL', 'BLNK', 'PEP',
       'CNP', 'MDT', 'GOOGL', 'ALNY', 'ALLY', 'RIVN', 'MMC', 'ATVI',
       'ABT', 'NFLX', 'CTAS', 'CDNA', 'BRO', 'GMED', 'RJF', 'PEG',
       'SBNY', 'SILK', 'EA', 'HPQ', 'Z', 'WAL', 'KMB', 'SHW', 'K', 'GO',
       'IONS', 'PAYC', 'PACW', 'MKC', 'ELAN', 'EW', 'ISRG', 'INCY',
       'IBRX', 'HPP', 'ATO', 'DEI', 'APPS', 'NLY', 'ED', 'BLK', 'BMRN',
       'BIIB', 'SRE', 'DHR', 'HE', 'CRM', 'SGEN', 'BATRK', 'CEG', 'CF',
       'RUN', 'UNICY', 'ELUXY', 'TOSYY', 'TELNY', 'SGAPY', 'KKOYY',
       'BRK-B', 'VFC', 'UNP', 'TTD', 'SYNH', 'SPWR', 'SWAV', 'SBCF',
       'RBLX', 'PG', 'NEE', 'LIN', 'INTC', 'HZNP', 'FATE', 'ENVX',
       'COST', 'CME', 'CHE', 'CELH', 'AVGO', 'ARWR', 'ABNB',
       'JAPSY', 'PSO']
assets.sort()

# Downloading data
og_data = yf.download(assets, start = start, end = end)
#data = og_data.loc[:,('Adj Close', slice(None))]
#data.columns = assets

#Y = data[assets].pct_change().dropna()
display(og_data)

display(start)
display(end)
#display(Y)

[*********************100%***********************]  691 of 691 completed


Adj Close                                                        \
                AAGIY         AAP        AAPL        AAT   AAVMY      ABBNY   
Date                                                                          
2022-08-15  39.549999  205.330002  173.190002  30.650000  10.760  30.590000   
2022-08-16  39.279999  209.600006  173.029999  30.570000  10.880  30.610001   
2022-08-17  38.939999  209.979996  174.550003  29.840000  10.670  30.030001   
2022-08-18  38.270000  207.490005  174.149994  30.160000  10.530  30.299999   
2022-08-19  38.049999  207.020004  171.520004  29.690001   9.845  29.719999   
...               ...         ...         ...        ...     ...        ...   
2023-08-08  36.720001   70.559998  179.800003  22.520000  16.230  39.060001   
2023-08-09  36.490002   71.199997  178.190002  22.379999  15.770  39.150002   
2023-08-10  37.799999   71.529999  177.970001  22.190001  15.690  38.250000   
2023-08-11  36.709999   72.279999  177.789993  22.280001  15.440  38.520000   
2023-08-14  36.389999   71.110001  179.460007  21.930000  15.300  38.610001   

                                                            ... Volume         \
                  ABBV        ABNB         ABT         ACN  ...  YAMHY  YARIY   
Date                                                        ...                 
2022-08-15  142.289993  126.040001  111.900002  320.329987  ...  17100  25000   
2022-08-16  142.550003  124.180000  112.029999  320.779999  ...  13000  20600   
2022-08-17  141.440002  121.870003  110.550003  318.450012  ...   2100   6800   
2022-08-18  141.289993  121.269997  109.959999  319.459991  ...   1200  18500   
2022-08-19  141.850006  114.760002  110.059998  315.290009  ...   4700   8700   
...                ...         ...         ...         ...  ...    ...    ...   
2023-08-08  149.619995  139.550003  105.580002  315.149994  ...   1500  28100   
2023-08-09  150.770004  137.869995  105.300003  311.529999  ...    200  50500   
2023-08-10  151.440002  135.970001  104.739998  310.410004  ...      0  31100   
2023-08-11  152.179993  133.679993  105.309998  309.049988  ...   1400  18300   
2023-08-14  152.270004  133.240005  106.099998  311.709991  ...      0  19900   

                                                                              
           YASKY      YOU     YUMC        Z  ZLNDY       ZS      ZTS   ZURVY  
Date                                                                          
2022-08-15  2300  2607800  1601700  2262700  29100  1131400  1243600   97000  
2022-08-16  4000  1573700  1448900  2059300  42700  1512700  2210700  101600  
2022-08-17  6500  1060500  1392300  3675900  54400  1583500  1367200   48600  
2022-08-18  1500  1619100  6319600  2847200  53300  1141000  1297900   73300  
2022-08-19  2000  1040900  1884800  3264100  43900  2554700  1223200   76700  
...          ...      ...      ...      ...    ...      ...      ...     ...  
2023-08-08  5800  1015200  1419100  2735900  14800  2532100  4118000   53100  
2023-08-09  2900  1041800  1983700  2327900  12900  1619000  2270000   57000  
2023-08-10  5600  1122000  1926400  2097700   6900  1886400  2348500   42200  
2023-08-11  5500  1162600  1642600  2064000  13400  1101000  1742900   67400  
2023-08-14  5200   964600  1480300  1871300   7800  1075600  1352400   57700  

[251 rows x 4146 columns]

'2022-08-15'

'2023-08-15'

In [11]:
data = og_data.loc[:,('Adj Close', slice(None))]
data.columns = assets

daily_returns = data[assets].pct_change().dropna()
display(daily_returns)

,AAGIY,AAP,AAPL,AAT,AAVMY,ABBNY,ABBV,ABNB,ABT,ACN,...,YAMHY,YARIY,YASKY,YOU,YUMC,Z,ZLNDY,ZS,ZTS,ZURVY
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-06,-0.045874,-0.016863,0.002509,-0.011783,-0.006369,-0.011573,-0.002689,-0.026805,-0.001583,-0.006228,...,0.000000,-0.007279,-0.029474,-0.015866,-0.026129,-0.038804,-0.039314,-0.035585,-0.005196,-0.011121
2023-07-07,0.010939,-0.008290,-0.005891,0.023328,0.032692,0.004258,-0.012750,0.016854,-0.007929,-0.007176,...,0.000000,0.017485,0.010389,0.001791,0.014601,0.005467,0.024220,0.010159,-0.007895,-0.000649
2023-07-10,-0.006039,-0.008360,-0.010856,0.011145,0.000497,0.001590,-0.007454,0.010666,0.006300,0.008798,...,-0.013966,0.023281,-0.037740,0.024139,0.013851,0.001882,0.014085,0.077455,0.018428,0.001082
2023-07-11,0.010127,0.019186,-0.002810,0.029058,0.014394,0.010053,0.008030,0.044264,0.004485,0.004117,...,-0.007082,0.016793,0.021959,0.093409,0.009936,0.091213,0.037500,-0.044597,-0.000470,0.006053
2023-07-12,0.024060,0.008985,0.008986,0.002434,0.023242,0.028811,-0.004352,-0.003781,-0.006047,0.013431,...,0.000000,0.026638,-0.002011,0.013174,0.039705,0.011859,0.111816,-0.065880,0.010109,0.015685
2023-07-13,0.023984,-0.001272,0.004057,0.013599,0.016139,0.006110,-0.010298,0.022405,0.008329,0.004237,...,0.000000,0.011936,0.024454,0.005516,0.024164,0.024007,0.009506,0.054184,-0.005004,0.009308
2023-07-14,-0.022706,-0.000283,0.000787,-0.003833,-0.010588,-0.002277,0.018115,0.023199,0.001764,0.001777,...,-0.000713,-0.009744,-0.018308,-0.019984,-0.009899,-0.017353,-0.012555,0.001998,0.006959,-0.005240
2023-07-17,0.001467,-0.016421,0.017306,0.002886,0.008918,-0.002790,-0.008676,0.014092,-0.008617,0.012699,...,0.000000,-0.017607,-0.000973,-0.019192,0.003999,0.009769,0.000636,0.043939,-0.008479,0.001264
2023-07-18,-0.008303,-0.002447,-0.001340,0.037410,0.007071,0.006867,0.012979,0.008049,-0.000467,0.008287,...,0.000000,0.032683,0.035467,-0.030982,-0.030042,0.012837,0.045108,0.019694,0.002050,0.001683


In [14]:
cumulative_returns = daily_returns.cumsum()
cumulative_returns

,AAGIY,AAP,AAPL,AAT,AAVMY,ABBNY,ABBV,ABNB,ABT,ACN,...,YAMHY,YARIY,YASKY,YOU,YUMC,Z,ZLNDY,ZS,ZTS,ZURVY
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-06,-0.045874,-0.016863,0.002509,-0.011783,-0.006369,-0.011573,-0.002689,-0.026805,-0.001583,-0.006228,...,0.000000,-0.007279,-0.029474,-0.015866,-0.026129,-0.038804,-0.039314,-0.035585,-0.005196,-0.011121
2023-07-07,-0.034935,-0.025154,-0.003383,0.011545,0.026323,-0.007315,-0.015439,-0.009951,-0.009512,-0.013404,...,0.000000,0.010206,-0.019084,-0.014075,-0.011527,-0.033337,-0.015095,-0.025426,-0.013091,-0.011769
2023-07-10,-0.040974,-0.033514,-0.014238,0.022690,0.026819,-0.005725,-0.022893,0.000715,-0.003213,-0.004606,...,-0.013966,0.033487,-0.056824,0.010065,0.002324,-0.031455,-0.001010,0.052029,0.005337,-0.010687
2023-07-11,-0.030848,-0.014328,-0.017048,0.051748,0.041213,0.004327,-0.014862,0.044978,0.001273,-0.000489,...,-0.021049,0.050280,-0.034866,0.103474,0.012260,0.059758,0.036490,0.007432,0.004867,-0.004635
2023-07-12,-0.006788,-0.005343,-0.008063,0.054183,0.064455,0.033138,-0.019215,0.041198,-0.004774,0.012943,...,-0.021049,0.076919,-0.036877,0.116647,0.051965,0.071617,0.148306,-0.058448,0.014976,0.011051
2023-07-13,0.017197,-0.006615,-0.004005,0.067781,0.080594,0.039248,-0.029512,0.063603,0.003556,0.017180,...,-0.021049,0.088854,-0.012422,0.122164,0.076129,0.095625,0.157811,-0.004264,0.009972,0.020359
2023-07-14,-0.005509,-0.006898,-0.003218,0.063948,0.070005,0.036971,-0.011397,0.086803,0.005319,0.018957,...,-0.021762,0.079111,-0.030730,0.102179,0.066229,0.078272,0.145256,-0.002266,0.016931,0.015119
2023-07-17,-0.004041,-0.023319,0.014088,0.066834,0.078923,0.034181,-0.020073,0.100895,-0.003298,0.031656,...,-0.021762,0.061503,-0.031703,0.082987,0.070228,0.088041,0.145892,0.041673,0.008452,0.016383
2023-07-18,-0.012344,-0.025766,0.012747,0.104244,0.085995,0.041048,-0.007094,0.108944,-0.003765,0.039943,...,-0.021762,0.094186,0.003764,0.052004,0.040187,0.100878,0.191000,0.061367,0.010502,0.018067
